# PC2Beam Demo - Hybrid Version (Works Locally & on Colab)

This notebook demonstrates the core functionality of the pc2beam package using the test dataset.

## Overview
The pc2beam package processes point clouds to extract beam-like structures by:
1. **S1 Features**: Computing supernormals (enhanced normal vectors)
2. **S2 Features**: Extracting segment orientation features
3. **Beam Projection**: Projecting points to beam model representation
4. **Visualization**: Interactive 3D and 2D visualizations

---
**Note**: This notebook automatically detects whether it's running locally or on Colab and adapts accordingly.

## Environment Detection and Setup

In [ ]:
# Detect if running on Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("🔄 Detected Google Colab environment")
except ImportError:
    IN_COLAB = False
    print("🔄 Detected local environment")

# Setup based on environment
if IN_COLAB:
    # Install package on Colab
    print("📦 Installing pc2beam from GitHub...")
    !pip install git+https://github.com/fnoi/pc2beam.git
    
    # Download test data
    import urllib.request
    import os
    os.makedirs('data', exist_ok=True)
    test_data_url = "https://raw.githubusercontent.com/fnoi/pc2beam/dev/data/test_points.txt"
    urllib.request.urlretrieve(test_data_url, "data/test_points.txt")
    print("✅ Test data downloaded")
    
    # Set data path for Colab
    data_path = "data/test_points.txt"
else:
    # Local environment setup
    import sys
    from pathlib import Path
    
    # Add project root to path
    project_root = Path.cwd().parent
    if str(project_root) not in sys.path:
        sys.path.append(str(project_root))
    
    # Set data path for local
    data_path = project_root / "data" / "test_points.txt"
    print(f"✅ Using local data: {data_path}")

print("✅ Environment setup complete!")

## Imports

In [ ]:
# Import required libraries
from pc2beam.data import PointCloud
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

print("✅ All imports successful!")

## Load Test Dataset

In [ ]:
# Load point cloud from test data
pc = PointCloud.from_txt(data_path)

print(f"✅ Loaded point cloud with {pc.size:,} points")
print(f"📊 Has normals: {pc.has_normals}")
print(f"🏷️  Has instance labels: {pc.has_instances}")
print(f"📏 Point cloud bounds: {pc.points.min(axis=0)} to {pc.points.max(axis=0)}")

## Visualize Original Point Cloud

In [ ]:
# Visualize original point cloud
fig_original = pc.visualize(
    color_by="instance",
    show_normals=True,
    normal_length=0.1,
    title="Original Point Cloud - Colored by Instance Labels",
    max_points=2000
)
fig_original.show()

## Compute S1 Features (Supernormals)

In [ ]:
# Compute S1 features (supernormals)
print("🔄 Computing S1 features (supernormals)...")
pc.calculate_s1(radius=0.1, use_radius=True)
print(f"✅ S1 features computed: {pc.has_s1_feature}")

# Visualize with supernormals
fig_s1 = pc.visualize_with_supernormals(
    color_by="instance",
    normal_length=0.1,
    title="Point Cloud with S1 Supernormals - Colored by Instance",
    max_points=2000
)
fig_s1.show()

## Compute S2 Features (Segment Orientation)

In [ ]:
# Compute S2 features (segment orientation)
print("🔄 Computing S2 features (segment orientation)...")
pc.calculate_s2(
    distance_threshold=0.01,
    ransac_n=3,
    num_iterations=1000
)
print(f"✅ S2 features computed: {pc.has_s2_feature}")

# Display S2 feature information
if pc.has_s2_feature:
    s2_instances = pc.s2_features.get_all_instances()
    print(f"📈 Found S2 features for {len(s2_instances)} instances")
    
    # Show first few S2 vectors
    for i, instance_id in enumerate(s2_instances[:3]):
        s2_vec = pc.s2_features.get_s2_vector(instance_id)
        line_pt = pc.s2_features.get_line_point(instance_id)
        print(f"   Instance {instance_id}: S2 vector = {s2_vec}, line point = {line_pt}")

## Project to Beam Model

In [ ]:
# Project points to beam model
print("🔄 Projecting points to beam model...")
pc.project_to_beam(min_points_per_instance=10)
print(f"✅ Beam projection completed: {pc.has_beam_projection}")

if pc.has_beam_projection:
    print(f"📊 Projected {len(pc.projected_points)} points to beam model")
    print(f"📏 Beam bounds: {pc.projected_points.min(axis=0)} to {pc.projected_points.max(axis=0)}")

## Visualize Results

In [ ]:
# Create side-by-side comparison
fig_comparison = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Original Point Cloud', 'Beam Projection'),
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]]
)

# Original point cloud
fig_orig = pc.visualize(
    color_by="instance",
    max_points=2000,
    show=False
)
fig_comparison.add_trace(fig_orig.data[0], row=1, col=1)

# Beam projection
fig_beam = pc.visualize_beam_projection(
    color_by="instance",
    max_points=2000,
    show=False
)
fig_comparison.add_trace(fig_beam.data[0], row=1, col=2)

fig_comparison.update_layout(
    title="Comparison: Original vs Beam Projection",
    height=500,
    showlegend=False
)
fig_comparison.show()

## 2D XY Projection View

In [ ]:
# Create 2D XY scatter plot of projected points
fig_xy = pc.visualize_projected_points_xy(
    title="Beam Projection - XY View (Top-Down)",
    color_by="instance",
    point_size=4,
    max_points=2000
)
fig_xy.show()

## Summary and Statistics

In [ ]:
# Display summary statistics
print("📋 PC2Beam Processing Summary")
print("=" * 40)
print(f"Environment: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"Input points: {pc.size:,}")
print(f"S1 features computed: {pc.has_s1_feature}")
print(f"S2 features computed: {pc.has_s2_feature}")
print(f"Beam projection: {pc.has_beam_projection}")

if pc.has_s2_feature:
    s2_instances = pc.s2_features.get_all_instances()
    print(f"S2 instances: {len(s2_instances)}")

if pc.has_beam_projection:
    print(f"Projected points: {len(pc.projected_points):,}")
    
    # Calculate compression ratio
    compression_ratio = len(pc.projected_points) / pc.size
    print(f"Compression ratio: {compression_ratio:.2%}")
    
    # Show unique instances in projection
    unique_instances = np.unique(pc.projected_instances)
    print(f"Unique instances in projection: {len(unique_instances)}")

print("✅ Processing complete!")
print(f"\n🎉 Demo completed successfully on {'Google Colab' if IN_COLAB else 'local environment'}!")